In [10]:
# 🧩 Exercise 1 — The 8-Puzzle and the Power of Heuristics

# --- Background ---
# The 8-puzzle consists of a 3x3 grid with tiles 1–8 and a blank space (0).
# Goal: Reach the ordered configuration 1–8 with the blank in the last cell.
# Moves: Slide one tile into the blank.
# We'll use A* with different heuristics and compare it with greedy best-first and hill climbing.


In [11]:
from heapq import heappush, heappop
import time

GOAL = (1,2,3,4,5,6,7,8,0)  # 0 = blank

# --- Utility: solvability check for 8-puzzle ---
# For 3x3, a state is solvable iff the inversion count is even (blank ignored).

def inversions(seq):
    arr = [x for x in seq if x != 0]
    inv = 0
    for i in range(len(arr)):
        for j in range(i+1, len(arr)):
            if arr[i] > arr[j]:
                inv += 1
    return inv


def is_solvable(state):
    return inversions(state) % 2 == 0


# --- Core helpers ---

def find_zero(s):
    return s.index(0)

def neighbors(s):
    """Return neighbor states reachable by sliding a tile into the blank."""
    i = find_zero(s)
    r, c = divmod(i, 3)
    swaps = []

    # TODO: Add valid swap indices for the blank (up, down, left, right)
    # Example: if r > 0: swaps.append(i-3)

    # up: i-3
    # down: i+3
    # right: i+1
    # left: i-1

    if r > 0:
        swaps.append(i-3)

    if c > 0:
        swaps.append(i-1)

    if r < 2:
        swaps.append(i+3)

    if c < 2:
        swaps.append(i+1)

    # TODO: For each valid swap j, create a new list from s and swap positions i and j
    for swap in swaps:
        new_list = []
        for x in s:
            new_list.append(x)

        tmp = new_list[i]
        new_list[i] = new_list[swap]
        new_list[swap] = tmp
        
        yield tuple(new_list), 1

In [12]:
def h_misplaced(s):
    """Number of misplaced tiles (excluding blank)."""
    h = 0
    target = GOAL 

    for i in range(len(GOAL)):
        if s[i] == 0 or target[i] == 0:
            continue

        elif s[i] != target[i]:
            h += 1

    return h

def h_manhattan(s):
    """Sum of Manhattan distances of tiles from goal positions."""
    distances = 0

    for i in range(len(GOAL)):
        pos_s = s.index(i)
        pos_ideal = GOAL.index(i)
        
        rx, cx = divmod(pos_s, 3) # get current position
        ry, cy = divmod(pos_ideal, 3) # get ideal position so to say

        distances += (abs(rx-ry) + abs(cx-cy))

    return distances    


In [13]:
def astar(start, h):
    """A* search using heuristic h."""
    explored = 0
    t0 = time.time()
    openq = []
    g = {start: 0}
    parent = {start: None}
    heappush(openq, (g[start] + h(start), g[start], start))
    closed = set()
    expanded = 0

    while openq:
        f, g_node, node = heappop(openq)

        if node in closed:
            continue 
        
        else:
            closed.add(node)

        if node == GOAL: 
            tf = time.time()

            results = {}
            results['cost'] = f
            results['time (s)'] = tf-t0

            walk = node 
            results['path'] = []

            while walk is not None: 
                # get solution
                results['path'].append(walk)
                walk = parent[walk] 
                

            results['expanded'] = explored

            results['path'] = tuple(results['path'][::-1])
            
            return results
        
        else:
            for child, cost in neighbors(node): 
                if child in closed:
                    continue 
                
                explored += 1
                g[child] = g_node + cost 
                parent[child] = node 
                heappush(openq, (g[child]+h(child), g[child], child))
    
    return None  # no solution found


In [14]:
# --- Pretty-print utilities (kept for clarity) ---
# Test stuff

def format_board(state):
    rows = []
    for r in range(3):
        row = state[3*r:3*(r+1)]
        rows.append(" ".join("__" if v==0 else f"{v:2d}" for v in row))
    return "\n".join(rows)


def moved_tile(prev_s, next_s):
    i0_prev = prev_s.index(0)
    i0_next = next_s.index(0)
    tile = next_s[i0_prev]
    r0p, c0p = divmod(i0_prev, 3)
    r0n, c0n = divmod(i0_next, 3)
    dr, dc = r0p - r0n, c0p - c0n
    if (dr, dc) == (1, 0): direction = "↓"
    elif (dr, dc) == (-1, 0): direction = "↑"
    elif (dr, dc) == (0, 1): direction = "→"
    elif (dr, dc) == (0, -1): direction = "←"
    else: direction = "?"
    return tile, direction


def print_solution(path):
    print(f"Solution with {len(path)-1} moves:\n")
    print("Step 0 (start):\n" + format_board(path[0]) + "\n")
    for k in range(1, len(path)):
        tile, direction = moved_tile(path[k-1], path[k])
        print(f"Step {k}: move tile {tile} {direction}")
        print(format_board(path[k]))
        print()


def print_astar_summary(label, res):
    if res is not None:
        summary = {k: v for k,v in res.items() if k != 'path'}
        print(f"{label}:", summary)
    else:
        print(f"{label}: failed to find a solution (unsolvable start or search issue).")


# --- Test cases ---
start = (2,8,3,1,6,5,7,0,4)                   # Original provided start
start_easy = (1,2,3,4,5,6,7,0,8)
start_unsolvable = (1,2,3,4,5,6,8,7,0)

# Warn for unsolvable puzzles
for name, s in [("original", start), ("easy", start_easy), ("unsolvable", start_unsolvable)]:
    if not is_solvable(s):
        print(f"[WARN] '{name}' start is not solvable (inversions={inversions(s)}). A* will return None.")


# --- Run A* and print summaries ---
res_mis = astar(start, h_misplaced)
print_astar_summary("A* (misplaced, original)", res_mis)
res_man = astar(start, h_manhattan)
print_astar_summary("A* (manhattan, original)", res_man)

res_easy = astar(start_easy, h_manhattan)
print_astar_summary("A* (manhattan, easy)", res_easy)



[WARN] 'unsolvable' start is not solvable (inversions=1). A* will return None.
A* (misplaced, original): {'cost': 15, 'time (s)': 0.0039942264556884766, 'expanded': 1157}
A* (manhattan, original): {'cost': 15, 'time (s)': 0.002002716064453125, 'expanded': 298}
A* (manhattan, easy): {'cost': 1, 'time (s)': 0.0, 'expanded': 3}


In [15]:
# --- Print solution steps ---
if res_man is not None and 'path' in res_man:
    print("\n--- Solution steps for A* (Manhattan, original) ---")
    print_solution(res_man['path'])
else:
    print("\nNo solution path to print for the original start.")



--- Solution steps for A* (Manhattan, original) ---
Solution with 15 moves:

Step 0 (start):
 2  8  3
 1  6  5
 7 __  4

Step 1: move tile 6 ↓
 2  8  3
 1 __  5
 7  6  4

Step 2: move tile 8 ↓
 2 __  3
 1  8  5
 7  6  4

Step 3: move tile 3 ←
 2  3 __
 1  8  5
 7  6  4

Step 4: move tile 5 ↑
 2  3  5
 1  8 __
 7  6  4

Step 5: move tile 4 ↑
 2  3  5
 1  8  4
 7  6 __

Step 6: move tile 6 →
 2  3  5
 1  8  4
 7 __  6

Step 7: move tile 8 ↓
 2  3  5
 1 __  4
 7  8  6

Step 8: move tile 4 ←
 2  3  5
 1  4 __
 7  8  6

Step 9: move tile 5 ↓
 2  3 __
 1  4  5
 7  8  6

Step 10: move tile 3 →
 2 __  3
 1  4  5
 7  8  6

Step 11: move tile 2 →
__  2  3
 1  4  5
 7  8  6

Step 12: move tile 1 ↑
 1  2  3
__  4  5
 7  8  6

Step 13: move tile 4 ←
 1  2  3
 4 __  5
 7  8  6

Step 14: move tile 5 ←
 1  2  3
 4  5 __
 7  8  6

Step 15: move tile 6 ↑
 1  2  3
 4  5  6
 7  8 __



In [16]:
# --- Greedy Best-First ---

def greedy_best_first(start, h):
    openq = [(h(start), start)]
    visited = set()
    expanded = 0

    while openq:
        _, u = heappop(openq)
        if u in visited: continue
        visited.add(u); expanded += 1
        if u == GOAL:
            return dict(expanded=expanded)
        for v,_ in neighbors(u):
            if v not in visited:
                heappush(openq, (h(v), v))

    return None


# --- Hill Climbing ---

def hill_climbing(start, h, max_steps=500):
    cur = start
    expanded = 0
    best_neighbour = None 
    best_cost = float("inf")
    cutoff = True 

    for _ in range(max_steps):
        cutoff = True 
        for neighbor, _ in neighbors(cur):
            expanded += 1

            # look for the best neighbour
            if (h_ := h(neighbor)) < best_cost: 
                best_neighbour = neighbor
                best_cost = h_


        if best_cost < h(cur):
            cur = best_neighbour
            cutoff = False

        if cur == GOAL:
            return {'found': True, "expanded": expanded, "solution": cur }

        if cutoff:
            break

    return dict(found=False, expanded=expanded)



In [17]:
# Baselines
print("Greedy (original):", greedy_best_first(start, h_manhattan))
print("Hill climbing (easy):", hill_climbing(start_easy, h_manhattan))
print("Hill climbing (original):", hill_climbing(start, h_manhattan))


Greedy (original): {'expanded': 53}
Hill climbing (easy): {'found': True, 'expanded': 3, 'solution': (1, 2, 3, 4, 5, 6, 7, 8, 0)}
Hill climbing (original): {'found': False, 'expanded': 5}
